# SETUPS

In [1]:
from functools import reduce
from collections import namedtuple
from copy import copy
from itertools import product
from random import random, randint, shuffle, seed, choice
import numpy as np
from scipy import sparse

def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points*2654435761+num_sets+density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True
    return sets

In [2]:
num_points=100
num_sets=num_points
x = make_set_covering_problem(num_points, num_sets, .3)
counter=0
print("Element at row=42 and column=42:", x[42,42])

Element at row=42 and column=42: True


In [3]:
def fitness(state, prob_size, SETS):
    cost = sum(state)
    valid = np.sum(
        reduce(
            np.logical_or,
            [SETS[i] for i, t in enumerate(state) if t],
            np.array([False for _ in range(prob_size)]),
        )
    )
    return valid, -cost

def tweak(state, prob_size):
    new_state = copy(state)
    index = randint(0, prob_size - 1)
    new_state[index] = not new_state[index]
    return new_state

# HILL CLIMBING

In [4]:
def hill_climbing(current_state, prob_size, SETS):
    best_solution = current_state
    best_valid, best_cost = fitness(best_solution, prob_size, SETS)
    evaluations = 1
    step = 0

    while True:
        step += 1
        neighbors = [tweak(best_solution, prob_size) for _ in range(prob_size)]
        found_better = False

        for neighbor in neighbors:
            neighbor_valid, neighbor_cost = fitness(neighbor, prob_size, SETS)

            if neighbor_valid > best_valid or (neighbor_valid == best_valid and neighbor_cost > best_cost):
                best_solution = neighbor
                best_valid = neighbor_valid
                best_cost = neighbor_cost
                found_better = True

            evaluations += 1

        if not found_better:
            break

    return evaluations, step, best_valid, best_cost


When using **hill climbing algorithm** for set covering problem, you can do the following steps:


1.   initialize the the best solution as the current state
2.   call the fitness function for the current state and compute the costs
3.   construct the neighbors using the tweak function
4.   compute the fitness function for each neighbor

      1.   if the fitness value of one of the neighbor is higher than the fitness value of the current state, then the new solution is updated
      2.   if the fitness value of one of the neighbors is not improving the current solution, then look at the costs: if the cost is updated, then update also the current solution

5. for each of this step, increase the number of steps and the number of evaluations





In [5]:
PROBLEM_SIZE = [100, 1_000, 5_000]
NUM_SETS = [100, 1_000, 5_000]
DENSITY = [.3, .7]

for prob_size in PROBLEM_SIZE:
    print("Problem size: ", prob_size)
    for density in DENSITY:
        print("Density: ", density)
        SETS = make_set_covering_problem(prob_size, prob_size, density).toarray()
        State = namedtuple('State', ['taken', 'not_taken'])
        current_state = [choice([False, False, False, False, False, False]) for _ in range(prob_size)]
        tot = hill_climbing(current_state, prob_size, SETS)
        print(f"At step {tot[1]} with {tot[0]} iterations of fitness function, the fitness of best solution is {tot[2]}")

Problem size:  100
Density:  0.3
At step 7 with 701 iterations of fitness function, the fitness of best solution is 100
Density:  0.7
At step 4 with 401 iterations of fitness function, the fitness of best solution is 100
Problem size:  1000
Density:  0.3
At step 11 with 11001 iterations of fitness function, the fitness of best solution is 1000
Density:  0.7
At step 5 with 5001 iterations of fitness function, the fitness of best solution is 1000
Problem size:  5000
Density:  0.3
At step 15 with 75001 iterations of fitness function, the fitness of best solution is 5000
Density:  0.7
At step 6 with 30001 iterations of fitness function, the fitness of best solution is 5000


# TABU' SEARCH

In [18]:
from queue import PriorityQueue
def tabu_search(current_state, prob_size, SETS, tabu_list_size):
    best_solution = current_state
    best_valid, best_cost = fitness(best_solution, prob_size, SETS)
    evaluations = 1
    step = 0
    tabu_list = PriorityQueue()

    while True:
        step += 1
        neighbors = [tweak(best_solution, prob_size) for _ in range(prob_size)]
        found_better = False

        for neighbor in neighbors:
            neighbor_valid, neighbor_cost = fitness(neighbor, prob_size, SETS)

            if neighbor not in tabu_list.queue and (neighbor_valid > best_valid or (neighbor_valid == best_valid and neighbor_cost > best_cost)):
                best_solution = neighbor
                best_valid = neighbor_valid
                best_cost = neighbor_cost
                found_better = True

            evaluations += 1

            tabu_list.put((neighbor_cost, neighbor))

            if len(tabu_list.queue) > tabu_list_size:
                tabu_list.get()

        if not found_better:
            break

    return evaluations, step, best_valid, best_cost


when using the tabu search algorithm for the ste covering problem, you can do the following steps:

1.   initialize the the best solution as the current state
2.   call the fitness function for the current state and compute the costs
3.   construct the neighbors using the tweak function
4.   compute the fitness function for each neighbor
5.   if the algorithm finds a neighbor solution better than the current one, then add it to the tabu list, which is ordered by cost, and update the solution
6. finally, return the best solution and the best cost, i.e., the solution with higher cost
7.   compute the number of evaluations and the number of steps increasing the respective counter at each iteration

In [19]:
PROBLEM_SIZE = [100, 1_000, 5_000]
NUM_SETS = [100, 1_000, 5_000]
DENSITY = [.3, .7]

for prob_size in PROBLEM_SIZE:
    print("Problem size: ", prob_size)
    for density in DENSITY:
        print("Density: ", density)
        SETS = make_set_covering_problem(prob_size, prob_size, density).toarray()
        State = namedtuple('State', ['taken', 'not_taken'])
        current_state = [choice([False, False, False, False, False, False]) for _ in range(prob_size)]
        tot = tabu_search(current_state, prob_size, SETS, 500)
        print(f"At step {tot[1]} with {tot[0]} iterations of fitness function, the fitness of best solution is {tot[2]}")

Problem size:  100
Density:  0.3
At step 7 with 701 iterations of fitness function, the fitness of best solution is 100
Density:  0.7
At step 4 with 401 iterations of fitness function, the fitness of best solution is 100
Problem size:  1000
Density:  0.3
At step 11 with 11001 iterations of fitness function, the fitness of best solution is 1000
Density:  0.7
At step 5 with 5001 iterations of fitness function, the fitness of best solution is 1000
Problem size:  5000
Density:  0.3
At step 15 with 75001 iterations of fitness function, the fitness of best solution is 5000
Density:  0.7
At step 6 with 30001 iterations of fitness function, the fitness of best solution is 5000


# SIMULATED ANNEALING

In [22]:
def simulated_annealing(current_state, prob_size, SETS, initial_temperature, cooling_rate):
    current_solution = current_state
    best_solution = current_solution
    current_valid, current_cost = fitness(current_solution, prob_size, SETS)
    best_valid, best_cost = current_valid, current_cost
    evaluations = 1
    step = 0


    while initial_temperature > 0.0000001:
        step += 1
        neighbor_solution = tweak(current_solution, prob_size)
        neighbor_valid, neighbor_cost = fitness(neighbor_solution, prob_size, SETS)

        delta_valid = neighbor_valid - current_valid
        delta_cost = neighbor_cost - current_cost

        if delta_valid > 0 or (delta_valid == 0 and delta_cost > 0) or (random() < np.exp((delta_valid + delta_cost) / initial_temperature)):
            current_solution = neighbor_solution
            current_valid = neighbor_valid
            current_cost = neighbor_cost

        evaluations += 1

        initial_temperature -= cooling_rate

    return evaluations, step, best_valid, best_cost


when using simulated annealing for set covering problem, you can do the following steps:

1.   initialize the the best solution as the current state
2.   call the fitness function for the current state and compute the costs
3.   construct the neighbors using the tweak function
4.   compute the fitness function for each neighbor
5.   to check if the new solution is better than the current one, compute the difference between the two fitness values and the two costs
6.   if these difference are positive or a random generated number is less than the worst solution, then the new solution is better than the current one so you can update it
7.   each iteration is given by the updating rate, so for each iteration two counters take note of the number of evaluations and the number of steps


In [23]:
PROBLEM_SIZE = [100, 1_000, 5_000]
NUM_SETS = [100, 1_000, 5_000]
DENSITY = [.3, .7]

for prob_size in PROBLEM_SIZE:
    print("Problem size: ", prob_size)
    for density in DENSITY:
        print("Density: ", density)
        SETS = make_set_covering_problem(prob_size, prob_size, density).toarray()
        State = namedtuple('State', ['taken', 'not_taken'])
        current_state = [choice([False, False, False, False, False, False]) for _ in range(prob_size)]
        tot = simulated_annealing(current_state, prob_size, SETS, 10, 0.01)
        print(f"At step {tot[1]} with {tot[0]} iterations of fitness function, the fitness of best solution is {tot[2]}")

Problem size:  100
Density:  0.3
At step 1000 with 1001 iterations of fitness function, the fitness of best solution is 0
Density:  0.7
At step 1000 with 1001 iterations of fitness function, the fitness of best solution is 0
Problem size:  1000
Density:  0.3
At step 1000 with 1001 iterations of fitness function, the fitness of best solution is 0
Density:  0.7
At step 1000 with 1001 iterations of fitness function, the fitness of best solution is 0
Problem size:  5000
Density:  0.3
At step 1000 with 1001 iterations of fitness function, the fitness of best solution is 0
Density:  0.7
At step 1000 with 1001 iterations of fitness function, the fitness of best solution is 0
